In [2]:
#Input

filtered_count_data = '../nf_output/data/Visium_FFPE_V44L01-325_A_filtered.csv'
spatial_data = '../nf_output/data/Visium_FFPE_V44L01-325_A_annotation.csv'
sample_name = 'Visium_FFPE_V44L01-325_A'
r_path = '/Library/Frameworks/R.framework/Resources'
min_clust = 11
max_clust = 15
method = 'mclust'
refine = 'false'

In [3]:
import torch
import re
import os
import sys
sys.path.append('GraphST_clustering')
from graphST_clustering_functions import *
from GraphST.GraphST import GraphST

In [4]:
#Output
sample = os.path.basename(filtered_count_data)
parts = sample.split('_')
last_char = parts[-1].split('.')[0]
if last_char.isdigit() == True: #simulation
    seed = last_char
    graphST_output = f"{sample_name}_sim_{seed}_clustered.h5ad"
else:
    graphST_output = f"{sample_name}_ref_clustered.h5ad"

In [5]:
# Run device, by default, the package is implemented on 'cpu'. We recommend using GPU.
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda:1')
else:
    device = torch.device('cpu')

# the location of R, which is necessary for mclust algorithm. Please replace the path below with local R installation path
os.environ["R_HOME"] = r_path

In [6]:
data, col_label = data_preprocessing(filtered_count_data, spatial_data)

In [7]:
model = GraphST.GraphST(data, device=device, random_seed = 1)
# train model
data = model.train()

Begin to train ST data...


100%|██████████| 600/600 [09:03<00:00,  1.10it/s]


Optimization finished for ST data!


In [8]:
# clustering 
method_list = method.replace(' ','').split(',')
data = graphST_methods_loop(data, method_list, list(range(min_clust,max_clust+1)), start=0.01, end=2, refinement=refine)

R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.1.1
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%


In [9]:
data.write_h5ad(graphST_output)